## Convert Nemo checkpoint to Huggingace format for Inferencing

The output from the training job is saved as nemo checkpoint. In this notebook we will convert the nemo checkpoint into a .pt weights file which can be used for inferencing.

To begin with we will retrieve the path for the checkpoint from the model output and also path to llama7b config file, this can be retreived from Notebook 3. Prepare Dataset.ipynb job output.

In [ ]:
nemo_checkpoint_path = "<Add s3 path from Training Job checkpoints>" # Checkpoint is saved as part of Notebook 5
path_to_llama_config = "s3://<pathto llama config>/config.json"  # this is saved as part of Notebook 3 output

In [ ]:
#retrive the docker image URL stored in step 1
%store -r docker_image 

In [ ]:
import sagemaker 

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

role = sagemaker.get_execution_role()

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

In [ ]:
hyperparameters = {}
hyperparameters["path_to_checkpoints"] = "/opt/ml/input/data/checkpoint"
hyperparameters["config_file"] = "/opt/ml/input/data/config/config.json"
hyperparameters["output_path"] = "/opt/ml/model"
hyperparameters["is_xser"] = True

In [ ]:
from sagemaker.pytorch import PyTorch
# Need to check if this works on multinode with torchrun.
estimator = PyTorch(
    base_job_name="nemo-megatron-data-prep",
    source_dir="./scripts",
    entry_point="convert_nemo_checkpoint_to_hf.py",
    role=role,
    image_uri=docker_image,
    instance_count=1,
    instance_type="ml.m5.4xlarge",
    sagemaker_session=sess,
    volume_size=512,
    hyperparameters=hyperparameters,
    debugger_hook_config=False,
    disable_output_compression=True,
    keep_alive_period_in_seconds=600,
)

In [ ]:
estimator.fit({"checkpoint":nemo_checkpoint_path,"config":path_to_llama_config})